In [1]:
import pandas as pd
import numpy as np
import ast

In [142]:
# Тренировочный датасет
data = pd.read_csv('data/train_data.csv')

data.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"['Начать бизнес', 'Самозанятые', 'Свое дело', ...",## Кто такой самозанятый?\n\nПо закону самозан...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","['Защитить права', 'Банки', 'Банковская карта'...",Первым делом нужно попросить банк проверить ма...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"['Кредиты', 'Долги', 'Просрочки', 'Ипотека', '...",## Не переоценивайте свои финансовые возможнос...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","['Инвестиции', 'Ценные бумаги', 'Фондовая бирж...",Просто прийти на биржу и купить ценные бумаги ...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"['Займы', 'Долги', 'Риски', 'Защитить права']","## МФО больше нет в госреестре. Значит, она за..."


In [143]:
df = data.copy()

## Этап 1: препроцессинг до чанкования

In [144]:
def first_preprocess(df):
    # annotation
    df['annotation'] = df['annotation'].apply(lambda x: '' if x is np.nan else x)

    # tags
    df['tags'] = [row[1:-1].replace("'", "") for row in df['tags']]

    # text
    df['text'] = df['text'].str.replace(r'Обновлено \d{2}\.\d{2}\.\d{4} в \d{2}:\d{2}', '', regex=True)

    return df

In [145]:
df = first_preprocess(df)

df.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"Начать бизнес, Самозанятые, Свое дело, Налоги",## Кто такой самозанятый?\n\nПо закону самозан...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","Защитить права, Банки, Банковская карта, Риски...",Первым делом нужно попросить банк проверить ма...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"Кредиты, Долги, Просрочки, Ипотека, Кредитная ...",## Не переоценивайте свои финансовые возможнос...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","Инвестиции, Ценные бумаги, Фондовая биржа, Акц...",Просто прийти на биржу и купить ценные бумаги ...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"Займы, Долги, Риски, Защитить права","## МФО больше нет в госреестре. Значит, она за..."


## Этап 2: Обработка text

Есть ответы которые содержат подпункты, начинающиеся с ### - в первой версии оставим это как часть ответа на вопрос ##

In [146]:
df['text'] = df['text'].str.replace('### ', ' ')

df.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"Начать бизнес, Самозанятые, Свое дело, Налоги",## Кто такой самозанятый?\n\nПо закону самозан...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","Защитить права, Банки, Банковская карта, Риски...",Первым делом нужно попросить банк проверить ма...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"Кредиты, Долги, Просрочки, Ипотека, Кредитная ...",## Не переоценивайте свои финансовые возможнос...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","Инвестиции, Ценные бумаги, Фондовая биржа, Акц...",Просто прийти на биржу и купить ценные бумаги ...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"Займы, Долги, Риски, Защитить права","## МФО больше нет в госреестре. Значит, она за..."


Разобьем каждый текст на блоки вопрос + ответ

In [99]:
def split_text(row:str):
    '''Функция сплитует по вопросу'''
    chunks = row.split('##')
    chunks = [chunk for chunk in chunks if chunk != '']
    return chunks

In [147]:
df['text'] = df['text'].apply(lambda x: split_text(x))

df.head()

,id,annotation,tags,text
0,doc_001,Светлана из Казани дает частные уроки английск...,"Начать бизнес, Самозанятые, Свое дело, Налоги",[ Кто такой самозанятый?\n\nПо закону самозаня...
1,doc_002,"Елене назначили социальное пособие на ребенка,...","Защитить права, Банки, Банковская карта, Риски...",[Первым делом нужно попросить банк проверить м...
2,doc_003,Самый надежный способ не оказаться в долгах — ...,"Кредиты, Долги, Просрочки, Ипотека, Кредитная ...",[ Не переоценивайте свои финансовые возможност...
3,doc_004,"Друзья Александра то и дело хвастаются, что по...","Инвестиции, Ценные бумаги, Фондовая биржа, Акц...",[Просто прийти на биржу и купить ценные бумаги...
4,doc_005,Вы взяли в микрофинансовой организации заем на...,"Займы, Долги, Риски, Защитить права","[ МФО больше нет в госреестре. Значит, она зак..."


Найдем вопрос в блоке и подствим его для каждого из строчек (todo тут стратегию чанкинга добавить)

In [163]:
example = df['text'][0][0]
print(example)

 Кто такой самозанятый?

По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль.

Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины.

Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.




In [164]:
candidats = example.split('\n\n')

if '?' in candidats[0]:
    question = candidats[0]
    example = candidats[1:]
else:
    question = ''

In [165]:
candidats

[' Кто такой самозанятый?',
 'По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль.',
 'Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины.',
 'Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.',
 '']

In [166]:
example

['По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль.',
 'Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины.',
 'Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.',
 '']

In [167]:
question

' Кто такой самозанятый?'

In [168]:
new_example = []
question = question.strip()

if question:
    for text in example:
        if text:
            text = text.strip()
            new_example.append(f"{question} {text}")
else:
    new_example = example

new_example

['Кто такой самозанятый? По закону самозанятый — это человек, который платит специальный **налог на профессиональный доход** (НПД). При этом не нужно дополнительно отчислять подоходный налог или налог на прибыль.',
 'Кто такой самозанятый? Получить статус самозанятого могут россияне и проживающие в РФ граждане Армении, Казахстана, Киргизии, Беларуси и Украины.',
 'Кто такой самозанятый? Оформить самозанятость вправе даже подростки с 14 лет, если они получили согласие родителей.']

In [176]:
# Пример без вопроса

example = df['text'][3][0]
print(example)

Просто прийти на биржу и купить ценные бумаги невозможно. Александру обязательно понадобится профессиональный посредник — брокер или доверительный управляющий, у которых есть лицензия Банка России.

Брокер совершает сделки по указаниям клиента. В таком случае именно Александр должен будет следить за ситуацией на бирже и принимать решения, когда и какие ценные бумаги покупать и в какой момент их продавать. О других участниках рынка ценных бумаг, без которых инвестору не обойтись, и особенностях их работы читайте в статье «Как устроена фондовая биржа».

Доверительный управляющий самостоятельно распоряжается деньгами или имуществом клиента, чтобы извлечь как можно больше прибыли для их владельца. При таком варианте Александру достаточно определиться, какую сумму он планирует вложить и на какой срок, сколько хотел бы заработать и к каким потерям готов. А затем просто периодически контролировать состояние своего инвестиционного портфеля. Сделки за него будет проводить управляющий.

Еще одно

In [177]:
candidats = example.split('\n\n')

if '?' in candidats[0]:
    question = candidats[0]
    example = candidats[1:]
else:
    question = ''

In [178]:
candidats

['Просто прийти на биржу и купить ценные бумаги невозможно. Александру обязательно понадобится профессиональный посредник — брокер или доверительный управляющий, у которых есть лицензия Банка России.',
 'Брокер совершает сделки по указаниям клиента. В таком случае именно Александр должен будет следить за ситуацией на бирже и принимать решения, когда и какие ценные бумаги покупать и в какой момент их продавать. О других участниках рынка ценных бумаг, без которых инвестору не обойтись, и особенностях их работы читайте в статье «Как устроена фондовая биржа».',
 'Доверительный управляющий самостоятельно распоряжается деньгами или имуществом клиента, чтобы извлечь как можно больше прибыли для их владельца. При таком варианте Александру достаточно определиться, какую сумму он планирует вложить и на какой срок, сколько хотел бы заработать и к каким потерям готов. А затем просто периодически контролировать состояние своего инвестиционного портфеля. Сделки за него будет проводить управляющий.',

In [179]:
example

'Просто прийти на биржу и купить ценные бумаги невозможно. Александру обязательно понадобится профессиональный посредник — брокер или доверительный управляющий, у которых есть лицензия Банка России.\n\nБрокер совершает сделки по указаниям клиента. В таком случае именно Александр должен будет следить за ситуацией на бирже и принимать решения, когда и какие ценные бумаги покупать и в какой момент их продавать. О других участниках рынка ценных бумаг, без которых инвестору не обойтись, и особенностях их работы читайте в статье «Как устроена фондовая биржа».\n\nДоверительный управляющий самостоятельно распоряжается деньгами или имуществом клиента, чтобы извлечь как можно больше прибыли для их владельца. При таком варианте Александру достаточно определиться, какую сумму он планирует вложить и на какой срок, сколько хотел бы заработать и к каким потерям готов. А затем просто периодически контролировать состояние своего инвестиционного портфеля. Сделки за него будет проводить управляющий.\n\nЕ

In [173]:
question

''

In [175]:
new_example = []
question = question.strip()

if question:
    for text in example:
        if text:
            text = text.strip()
            new_example.append(f"{question} {text}")
else:
    new_example = example

new_example

'Просто прийти на биржу и купить ценные бумаги невозможно. Александру обязательно понадобится профессиональный посредник — брокер или доверительный управляющий, у которых есть лицензия Банка России.\n\nБрокер совершает сделки по указаниям клиента. В таком случае именно Александр должен будет следить за ситуацией на бирже и принимать решения, когда и какие ценные бумаги покупать и в какой момент их продавать. О других участниках рынка ценных бумаг, без которых инвестору не обойтись, и особенностях их работы читайте в статье «Как устроена фондовая биржа».\n\nДоверительный управляющий самостоятельно распоряжается деньгами или имуществом клиента, чтобы извлечь как можно больше прибыли для их владельца. При таком варианте Александру достаточно определиться, какую сумму он планирует вложить и на какой срок, сколько хотел бы заработать и к каким потерям готов. А затем просто периодически контролировать состояние своего инвестиционного портфеля. Сделки за него будет проводить управляющий.\n\nЕ